In [3]:
import norgatedata

print("NDU running:", norgatedata.status())
syms = set(norgatedata.futures_market_symbols())

# Check a few likely micro FX codes
for s in ["M6A", "M6E", "M6B", "6A", "6E", "6B"]:
    print(s, s in syms)


NDU running: True
M6A False
M6E False
M6B False
6A True
6E True
6B True


In [4]:
"""
Builds a comprehensive futures contract universe from Norgate Data.
Downloads all available futures markets, retrieves contract specifications,
classifies by sector, and creates metadata for both continuous and individual contracts.

Outputs:
- ./01-futures_universe/futures_contracts_full.parquet - Complete contract database
- ./01-futures_universe/futures_contracts_full.csv - CSV version for easy viewing
- ./01-futures_universe/continuous_contracts.csv - Continuous contract list for backtesting
- system_verification/FUTURES_UNIVERSE_BUILDER/ - Timestamped diagnostics

Requirements:
- Norgate Data Updater (NDU) running
- norgatedata package installed
- Optional: margin_requirements.csv for broker margins
"""

import os
from datetime import datetime
import pandas as pd
import numpy as np
import norgatedata
from typing import Dict, List, Optional, Tuple
from dataclasses import dataclass, asdict


RUN_TIMESTAMP = datetime.now().strftime('%Y%m%d-%H%M%S')


# ============================================================
# PRETTY COLORS
# ============================================================
class C:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    BOLD = '\033[1m'
    ENDC = '\033[0m'


# ============================================================
# CONFIG
# ============================================================
BASE_DIR = "./01-futures_universe"
os.makedirs(BASE_DIR, exist_ok=True)

FULL_PARQUET_PATH = os.path.join(BASE_DIR, "futures_contracts_full.parquet")
FULL_CSV_PATH = os.path.join(BASE_DIR, "futures_contracts_full.csv")
CONTINUOUS_CSV_PATH = os.path.join(BASE_DIR, "continuous_contracts.csv")
TRADEABLE_CSV_PATH = os.path.join(BASE_DIR, "tradeable_universe.csv")

VERIFICATION_DIR = "system_verification/01-FUTURES_UNIVERSE_BUILDER"
os.makedirs(VERIFICATION_DIR, exist_ok=True)

# Optional: External margin file
MARGIN_FILE_PATH = "./External_margin_file/margin_requirements.csv"
MARGIN_COLUMN = "Overnight Maintenance"

# Contract details spreadsheet (Norgate reference)
CONTRACT_DETAILS_PATH = "./futures_contract_details/Futures_Contract_Details.csv"# ============================================================
# CONTRACT SIZE REFERENCE (from spreadsheet)
# ============================================================
CONTRACT_UNITS = {
    # Currencies
    '6A': '100,000 AUD', '6B': '62,500 GBP', '6C': '100,000 CAD',
    '6E': '125,000 EUR', '6J': '12,500,000 JPY', '6M': '500,000 MXN',
    '6N': '100,000 NZD', '6S': '125,000 CHF', 'DX': '$1,000 x Index',
    'BTC': '5 bitcoin', 'MBT': '0.1 bitcoin', 'ETH': '50 Ether', 'MET': '0.1 Ether',
    
    # Grains & Oilseeds
    'ZC': '5,000 bushels', 'ZW': '5,000 bushels', 'ZS': '5,000 bushels',
    'ZM': '100 tons', 'ZL': '60,000 lbs', 'ZO': '5,000 bushels',
    'ZR': '2,000 cwt', 'KE': '5,000 bushels',
    
    # Softs
    'CC': '10 tonnes', 'KC': '37,500 lbs', 'SB': '112,000 lbs',
    'CT': '50,000 lbs', 'OJ': '15,000 lbs', 'LRC': '10 tonnes',
    
    # Livestock
    'LE': '40,000 lbs', 'HE': '40,000 lbs', 'GF': '50,000 lbs',
    'DC': '200,000 lbs',
    
    # Energy
    'CL': '1,000 barrels', 'BRN': '1,000 barrels', 'NG': '10,000 MMBtu',
    'RB': '42,000 gallons', 'HO': '42,000 gallons', 'GWM': '1,000 therms',
    
    # Metals
    'GC': '100 oz', 'SI': '5,000 oz', 'HG': '25,000 lbs',
    'PL': '50 oz', 'PA': '100 oz',
    
    # Stock Indices
    'ES': '$50 x Index', 'MES': '$5 x Index', 'NQ': '$20 x Index',
    'MNQ': '$2 x Index', 'RTY': '$50 x Index', 'M2K': '$5 x Index',
    'YM': '$5 x Index', 'MYM': '$0.50 x Index', 'EMD': '$100 x Index',
    'VX': '$1,000 x Index', 'FESX': 'EUR 10 x Index', 'FDAX': 'EUR 25 x Index',
    'NIY': '500 JPY x Index', 'NKD': '$5 x Index', 'YAP': 'AUD 25 x Index',
    
    # Interest Rates
    'ZB': '$100,000 face', 'ZN': '$100,000 face', 'ZF': '$100,000 face',
    'ZT': '$200,000 face', 'UB': '$100,000 face', 'TN': '$100,000 face',
    'ZQ': '$5,000,000', 'SR3': '$2,500 x Index', 'FGBL': 'EUR 100,000 face',
    'FGBM': 'EUR 100,000 face', 'FGBS': 'EUR 100,000 face',
    
    # Australian
    'AFB': '20 tonnes', 'AWM': '20 tonnes',
}
# ============================================================
# 1. CHECK NORGATE CONNECTION
# ============================================================
print(C.HEADER + "\n" + "=" * 70)
print("NORGATE FUTURES UNIVERSE BUILDER")
print("=" * 70 + C.ENDC)
print(f"\nRun timestamp: {RUN_TIMESTAMP}")

print("\n" + C.OKBLUE + "Checking Norgate Data connection..." + C.ENDC)

if not norgatedata.status():
    print(C.FAIL + "❌ ERROR: Norgate Data Updater (NDU) is not running!" + C.ENDC)
    print("Please start NDU and run this script again.")
    exit(1)

print(C.OKGREEN + "✓ Norgate Data Updater is running" + C.ENDC)


# ============================================================
# 2. LOAD EXTERNAL DATA SOURCES
# ============================================================
print("\n" + C.OKBLUE + "Loading external data sources..." + C.ENDC)

# Load contract details spreadsheet if available
contract_details = {}
if os.path.exists(CONTRACT_DETAILS_PATH):
    try:
        df_details = pd.read_csv(CONTRACT_DETAILS_PATH)
        for _, row in df_details.iterrows():
            symbol = str(row['Code']).strip().upper()
            contract_details[symbol] = {
                'name': row['Name'],
                'exchange': row['Exchange'],
                'group': row['Group'],
                'contract_size': row['Contract Size'],
                'quotation': row['Quotation'],
                'tick_size_desc': row['Tick Size'],
                'tick_value': row['Tick Value'],
                'point_value_ref': row['Point Value'],
                'currency': row['Currency'],
            }
        print(f"  ✓ Loaded {len(contract_details)} contract specifications from spreadsheet")
    except Exception as e:
        print(f"  ⚠️ Could not load contract details: {e}")
else:
    print(f"  ⚠️ Contract details file not found: {CONTRACT_DETAILS_PATH}")

# Load margin requirements if available
margin_lookup = {}
if os.path.exists(MARGIN_FILE_PATH):
    try:
        df_margins = pd.read_csv(MARGIN_FILE_PATH)
        # Sort to prefer USD
        df_margins['_is_usd'] = df_margins['Currency'] == 'USD'
        df_margins = df_margins.sort_values('_is_usd', ascending=False)
        
        for _, row in df_margins.iterrows():
            symbol = str(row['Trading Class']).strip().upper()
            if symbol in margin_lookup:
                continue
            try:
                margin_value = float(row[MARGIN_COLUMN])
                if pd.notna(margin_value) and margin_value > 0:
                    margin_lookup[symbol] = margin_value
            except (ValueError, TypeError):
                continue
        print(f"  ✓ Loaded {len(margin_lookup)} margin requirements from spreadsheet")
    except Exception as e:
        print(f"  ⚠️ Could not load margins: {e}")
else:
    print(f"  ⚠️ Margin file not found: {MARGIN_FILE_PATH}")


# ============================================================
# 3. GET ALL FUTURES MARKETS FROM NORGATE
# ============================================================
print("\n" + C.OKBLUE + "Fetching futures markets from Norgate..." + C.ENDC)

futures_markets = norgatedata.futures_market_symbols()
print(f"  Total futures markets available: {len(futures_markets)}")


# ============================================================
# 4. BUILD COMPREHENSIVE CONTRACT DATABASE
# ============================================================
print("\n" + C.OKBLUE + "Building contract database..." + C.ENDC)
print("-" * 70)

contracts_data = []
errors = []
margin_sources = {'spreadsheet': 0, 'norgate': 0, 'default': 0}

for i, symbol in enumerate(futures_markets, 1):
    continuous_symbol = f"&{symbol}"
    
    try:
        # Get metadata from Norgate
        market_name = norgatedata.futures_market_name(symbol) or symbol
        point_value = norgatedata.point_value(continuous_symbol)
        tick_size = norgatedata.tick_size(continuous_symbol)
        
        # Get sector classification
        sector = norgatedata.classification(
            continuous_symbol,
            'NorgateDataFuturesClassification',
            'Name'
        ) or 'Unknown'
        
        # Get margin (try spreadsheet first, then Norgate)
        margin = None
        margin_source = 'default'
        
        if symbol in margin_lookup:
            margin = margin_lookup[symbol]
            margin_source = 'spreadsheet'
            margin_sources['spreadsheet'] += 1
        else:
            try:
                norgate_margin = norgatedata.margin(continuous_symbol)
                if norgate_margin and norgate_margin > 0:
                    margin = norgate_margin
                    margin_source = 'norgate'
                    margin_sources['norgate'] += 1
            except:
                pass
        
        if margin is None:
            margin = 5000  # Default fallback
            margin_source = 'default'
            margin_sources['default'] += 1
        
        # Get contract unit description
        contract_unit = CONTRACT_UNITS.get(symbol, 'Unknown')
        
        # Get additional details from spreadsheet if available
        exchange = ''
        quotation = ''
        currency = 'USD'
        
        if symbol in contract_details:
            details = contract_details[symbol]
            exchange = details.get('exchange', '')
            quotation = details.get('quotation', '')
            currency = details.get('currency', 'USD')
            if contract_unit == 'Unknown':
                contract_unit = details.get('contract_size', 'Unknown')
        
        # Calculate tick value
        tick_value = tick_size * point_value if (tick_size and point_value) else None
        
        # Get data availability info
        try:
            test_data = norgatedata.price_timeseries(
                continuous_symbol,
                start_date='2000-01-01',
                format='pandas-dataframe'
            )
            if test_data is not None and len(test_data) > 0:
                data_start = test_data.index[0].strftime('%Y-%m-%d')
                data_end = test_data.index[-1].strftime('%Y-%m-%d')
                data_points = len(test_data)
                has_data = True
            else:
                data_start = None
                data_end = None
                data_points = 0
                has_data = False
        except:
            data_start = None
            data_end = None
            data_points = 0
            has_data = False
        
        contracts_data.append({
            'symbol': symbol,
            'continuous_symbol': continuous_symbol,
            'name': market_name,
            'sector': sector,
            'exchange': exchange,
            'currency': currency,
            'point_value': point_value,
            'tick_size': tick_size,
            'tick_value': tick_value,
            'margin': margin,
            'margin_source': margin_source,
            'contract_unit': contract_unit,
            'quotation': quotation,
            'has_data': has_data,
            'data_start': data_start,
            'data_end': data_end,
            'data_points': data_points,
        })
        
        if i % 20 == 0:
            print(f"  Processed {i}/{len(futures_markets)} markets...")
            
    except Exception as e:
        errors.append(f"{symbol}: {str(e)[:50]}")
        continue

print(f"\n✓ Built database with {len(contracts_data)} contracts")


# ============================================================
# 5. CREATE DATAFRAME AND SAVE
# ============================================================
print("\n" + C.OKBLUE + "Saving contract database..." + C.ENDC)

df_contracts = pd.DataFrame(contracts_data)

# Sort by sector, then by name
df_contracts = df_contracts.sort_values(['sector', 'name']).reset_index(drop=True)

# Save to Parquet
df_contracts.to_parquet(FULL_PARQUET_PATH, index=False)
print(f"  ✓ Saved Parquet → {FULL_PARQUET_PATH}")

# Save to CSV
df_contracts.to_csv(FULL_CSV_PATH, index=False)
print(f"  ✓ Saved CSV → {FULL_CSV_PATH}")


# ============================================================
# 6. CREATE CONTINUOUS CONTRACTS LIST FOR BACKTESTING
# ============================================================
print("\n" + C.OKBLUE + "Creating continuous contracts list..." + C.ENDC)

df_continuous = df_contracts[df_contracts['has_data'] == True].copy()
df_continuous = df_continuous[[
    'symbol', 'continuous_symbol', 'name', 'sector', 
    'point_value', 'margin', 'margin_source', 'contract_unit',
    'data_start', 'data_end', 'data_points'
]]

df_continuous.to_csv(CONTINUOUS_CSV_PATH, index=False)
print(f"  ✓ Saved {len(df_continuous)} continuous contracts → {CONTINUOUS_CSV_PATH}")


# ============================================================
# 7. CREATE TRADEABLE UNIVERSE (WITH FILTERS)
# ============================================================
print("\n" + C.OKBLUE + "Creating tradeable universe..." + C.ENDC)

# Filter criteria for a tradeable universe:
# - Has data
# - Has valid point value
# - USD currency (for simplicity)
# - Margin available

df_tradeable = df_contracts[
    (df_contracts['has_data'] == True) &
    (df_contracts['point_value'].notna()) &
    (df_contracts['point_value'] > 0) &
    (df_contracts['currency'].isin(['USD', 'EUR', 'GBP', 'AUD', 'CAD', 'CHF', 'JPY']))
].copy()

# Add notional value estimate (using recent price if available)
# For now, just flag contracts as tradeable
df_tradeable['tradeable'] = True

df_tradeable.to_csv(TRADEABLE_CSV_PATH, index=False)
print(f"  ✓ Saved {len(df_tradeable)} tradeable contracts → {TRADEABLE_CSV_PATH}")


# ============================================================
# 8. DIAGNOSTICS
# ============================================================
print(C.HEADER + "\n" + "=" * 70)
print("DIAGNOSTICS")
print("=" * 70 + C.ENDC)

# Summary by sector
print("\n" + C.OKCYAN + "Contracts by Sector:" + C.ENDC)
sector_summary = df_contracts.groupby('sector').agg({
    'symbol': 'count',
    'margin': ['min', 'max', 'mean'],
    'has_data': 'sum'
}).round(0)
sector_summary.columns = ['Count', 'Min Margin', 'Max Margin', 'Avg Margin', 'With Data']
print(sector_summary.to_string())

# Margin source summary
print("\n" + C.OKCYAN + "Margin Sources:" + C.ENDC)
print(f"  From spreadsheet: {margin_sources['spreadsheet']}")
print(f"  From Norgate:     {margin_sources['norgate']}")
print(f"  Default (5000):   {margin_sources['default']}")

# Data availability summary
with_data = df_contracts['has_data'].sum()
without_data = len(df_contracts) - with_data
print("\n" + C.OKCYAN + "Data Availability:" + C.ENDC)
print(f"  With data:    {with_data}")
print(f"  Without data: {without_data}")

# List contracts without data
no_data_contracts = df_contracts[df_contracts['has_data'] == False]['symbol'].tolist()
if no_data_contracts:
    print(f"\n" + C.WARNING + "Contracts without data:" + C.ENDC)
    print(f"  {no_data_contracts}")

# Errors
if errors:
    print(f"\n" + C.FAIL + f"Errors ({len(errors)}):" + C.ENDC)
    for err in errors[:10]:
        print(f"  - {err}")
    if len(errors) > 10:
        print(f"  ... and {len(errors) - 10} more")


# ============================================================
# 9. DETAILED CONTRACT LIST BY SECTOR
# ============================================================
print("\n" + C.HEADER + "=" * 70)
print("DETAILED CONTRACT LIST")
print("=" * 70 + C.ENDC)

for sector in sorted(df_contracts['sector'].unique()):
    sector_df = df_contracts[df_contracts['sector'] == sector]
    print(f"\n" + C.OKBLUE + f"{sector} ({len(sector_df)} contracts):" + C.ENDC)
    print("-" * 70)
    
    for _, row in sector_df.iterrows():
        data_indicator = "✓" if row['has_data'] else "✗"
        margin_str = f"${row['margin']:,.0f}" if row['margin'] else "N/A"
        pv_str = f"${row['point_value']:,.0f}" if row['point_value'] else "N/A"
        
        print(f"  {data_indicator} {row['symbol']:6} {row['name'][:30]:32} "
              f"PV: {pv_str:>10}  Margin: {margin_str:>10} ({row['margin_source']})")


# ============================================================
# 10. SAVE DIAGNOSTICS
# ============================================================
print("\n" + C.OKBLUE + "Saving diagnostics..." + C.ENDC)

diag_path = os.path.join(VERIFICATION_DIR, f"futures_universe_diagnostics-{RUN_TIMESTAMP}.csv")
df_contracts.to_csv(diag_path, index=False)
print(f"  ✓ Saved diagnostics → {diag_path}")

# Save summary statistics
summary_path = os.path.join(VERIFICATION_DIR, f"futures_universe_summary-{RUN_TIMESTAMP}.txt")
with open(summary_path, 'w') as f:
    f.write(f"Norgate Futures Universe Builder\n")
    f.write(f"Run timestamp: {RUN_TIMESTAMP}\n")
    f.write(f"=" * 70 + "\n\n")
    f.write(f"Total markets: {len(futures_markets)}\n")
    f.write(f"Contracts built: {len(contracts_data)}\n")
    f.write(f"With data: {with_data}\n")
    f.write(f"Without data: {without_data}\n")
    f.write(f"Tradeable: {len(df_tradeable)}\n\n")
    f.write(f"Margin sources:\n")
    f.write(f"  Spreadsheet: {margin_sources['spreadsheet']}\n")
    f.write(f"  Norgate: {margin_sources['norgate']}\n")
    f.write(f"  Default: {margin_sources['default']}\n\n")
    f.write(f"Sectors:\n")
    f.write(sector_summary.to_string())
print(f"  ✓ Saved summary → {summary_path}")


# ============================================================
# 11. FINAL OUTPUT
# ============================================================
print(C.HEADER + "\n" + "=" * 70)
print("COMPLETED")
print("=" * 70 + C.ENDC)

print(f"\n" + C.OKGREEN + "Output files:" + C.ENDC)
print(f"  📊 {FULL_PARQUET_PATH}")
print(f"  📊 {FULL_CSV_PATH}")
print(f"  📊 {CONTINUOUS_CSV_PATH}")
print(f"  📊 {TRADEABLE_CSV_PATH}")
print(f"  📋 {diag_path}")
print(f"  📋 {summary_path}")

print(f"\n" + C.OKGREEN + "Summary:" + C.ENDC)
print(f"  Total markets:     {len(futures_markets)}")
print(f"  Contracts built:   {len(contracts_data)}")
print(f"  With data:         {with_data}")
print(f"  Tradeable:         {len(df_tradeable)}")
print(f"  Sectors:           {len(df_contracts['sector'].unique())}")

print(C.HEADER + "\n========== DONE ==========" + C.ENDC)



NORGATE FUTURES UNIVERSE BUILDER

Run timestamp: 20260128-180320

Checking Norgate Data connection...
✓ Norgate Data Updater is running

Loading external data sources...
  ✓ Loaded 105 contract specifications from spreadsheet
  ✓ Loaded 349 margin requirements from spreadsheet

Fetching futures markets from Norgate...
  Total futures markets available: 105

Building contract database...
----------------------------------------------------------------------
  Processed 20/105 markets...
  Processed 40/105 markets...
  Processed 60/105 markets...
  Processed 80/105 markets...
  Processed 100/105 markets...

✓ Built database with 105 contracts

Saving contract database...
  ✓ Saved Parquet → ./01-futures_universe\futures_contracts_full.parquet
  ✓ Saved CSV → ./01-futures_universe\futures_contracts_full.csv

Creating continuous contracts list...
  ✓ Saved 105 continuous contracts → ./01-futures_universe\continuous_contracts.csv

Creating tradeable universe...
  ✓ Saved 101 tradeable cont